In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------
# Load Data
# ------------------------------
df = pd.read_csv("C:/Users/jay.jain/Desktop/Codes - Jay/Assignment 1/FEIcm2.csv")
df.columns = ['Ticker1', 'Ticker2', 'Description', 'Timestamp', 'Col5', 'Frequency', 
              'Open', 'High', 'Low', 'Close', 'Volume']

# Convert Timestamp to datetime and clean
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors='coerce')
df = df.dropna(subset=["Timestamp"])
df = df.sort_values("Timestamp")

# Filter for years 2012–2015
df = df[(df["Timestamp"].dt.year >= 2023) & (df["Timestamp"].dt.year <= 2024)]

# Set Timestamp as index
df.set_index("Timestamp", inplace=True)

# ------------------------------
# Resample to Weekly Frequency
# ------------------------------
weekly_df = df['Close'].resample('W-FRI').last()  # Use last closing price each week

# ------------------------------
# Compute Weekly Z-Score
# ------------------------------
window = 4  # 4 weeks ~ 1 month

rolling_mean = weekly_df.rolling(window=window).mean()
rolling_std = weekly_df.rolling(window=window).std()
z_score = (weekly_df - rolling_mean) / rolling_std

# ------------------------------
# Plot Weekly Price and Z-Score
# ------------------------------
plt.figure(figsize=(14, 8))

# Plot 1: Weekly Close and Moving Average
plt.subplot(2, 1, 1)
plt.plot(weekly_df.index, weekly_df, label='Weekly Close', color='blue')
plt.plot(rolling_mean.index, rolling_mean, label=f'{window}-Week MA', color='orange', linestyle='--')
plt.title('Weekly Close Price and Moving Average (2012–2015)')
plt.ylabel('Price')
plt.legend()

# Plot 2: Z-Score
plt.subplot(2, 1, 2)
plt.plot(z_score.index, z_score, label='Weekly Z-Score', color='purple')
plt.axhline(2, color='red', linestyle='--', label='Overbought (+2)')
plt.axhline(-2, color='green', linestyle='--', label='Oversold (-2)')
plt.axhline(0, color='black', linestyle=':')
plt.title('Z-Score of Weekly Close vs Moving Average')
plt.xlabel('Date')
plt.ylabel('Z-Score')
plt.legend()

plt.tight_layout()
plt.show()